In [11]:
import sys
import importlib

# Clear any cached modules
modules_to_clear = [m for m in list(sys.modules.keys()) if 'marooned' in m or m in ['environment', 'config', 'models', 'game_state', 'view_map']]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

# Now do fresh imports
sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from config import (
    ActionType, ResourceType, MapLevel,
    MAX_ENERGY, ENERGY_COST_WALK, ENERGY_COST_GATHER,
    ENERGY_COST_CLIMB_UP, ENERGY_COST_CLIMB_DOWN,
    BASE_CAMP_POSITION
)
from models import Action, Position
import random

print("✅ Modules loaded fresh (cache cleared)")

✅ Modules loaded fresh (cache cleared)


In [12]:
# Create environment with seed for reproducibility
env = MaroonedEnv(seed=42)
observations = env.reset()
PLAYER_ID = "Alice"

print("🏝️ Environment initialized!")
print(f"Traitor: {env.state.traitor_id}")
print(f"Starting position: {BASE_CAMP_POSITION}")
print(f"Map sizes: Ground(30x30), Mountain(10x10), Cave(15x15)")

# Helper function
def show_sailor_status():
    """Display current sailor status"""
    sailor = env.state.sailors[PLAYER_ID]
    backpack_dict = {}
    for item in sailor.backpack:
        backpack_dict[item.resource_type] = item.quantity
    
    print(f"📍 Position: {sailor.position.to_tuple()}")
    print(f"⚡ Energy: {sailor.energy}/{MAX_ENERGY}")
    print(f"🎒 Backpack: {backpack_dict}")
    print("-" * 50)

🏝️ Environment initialized!
Traitor: Bob
Starting position: (15, 15, <MapLevel.GROUND: 0>)
Map sizes: Ground(30x30), Mountain(10x10), Cave(15x15)


In [13]:
# Display all three levels
print(env.render_map(MapLevel.GROUND, use_emoji=True))
print(env.render_map(MapLevel.MOUNTAIN, use_emoji=True))
print(env.render_map(MapLevel.CAVE, use_emoji=True))


🏝️  GROUND LEVEL (Z=0)
Legend: 🟫 land | 🌲 wood | ⚙️ metal | 🍎 food | 🌿 antidote | ☠️ poison
        ⬆️ stairs up | ⬇️ stairs down | 🏠 base | A/B/C/D/E sailors | 5👥 group

   012345678901234567890123456789
 0 🟫🟫🟫🟫🍎🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫
 1 🍎🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫
 2 🟫🟫🟫🟫🌲🟫🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 3 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫⚙️🟫⚙️🟫⚙️🍎🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫
 4 🟫🟫🌲🟫🍎🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫
 5 🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🌲🟫🟫🟫
 6 🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🍎🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 7 🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🍎🟫🟫🟫🟫
 8 ⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🍎🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🌲
 9 🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🌲🌲
10 🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🍎🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
11 🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🌲🟫🟫🌲🟫
12 🟫🍎🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫🟫⚙️🟫🟫🟫🟫
13 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
14 🟫🟫🟫🌲🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫🟫
15 🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫5👥🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫
16 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🍎🍎⬇️🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
17 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫⚙️🟫
18 🟫🌲🟫🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
19 🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫⬆️🟫🟫🟫🟫🍎🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫⚙️
20 🟫🟫⚙️🟫🌲🟫🍎⚙️🟫⚙️🟫🟫🟫🍎☠️🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫⚙️🟫🟫🟫
21 🟫🟫🍎🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🌲🟫🟫🍎🟫🟫
22 🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫

In [14]:
# TEST: Navigate to stairs and climb to different levels

sailor = env.state.sailors[PLAYER_ID]
print(f"🧪 TEST: Multi-Level Navigation")
print(f"Starting position: {sailor.position.to_tuple()}\n")

# Find all stairs on ground level
stairs_info = []
for pos1, pos2 in env.state.world_map.level_transitions:
    if pos1.level == MapLevel.GROUND:
        direction = "UP" if pos2.level.value > pos1.level.value else "DOWN"
        stairs_info.append((pos1, direction, pos2))
    if pos2.level == MapLevel.GROUND:
        direction = "UP" if pos1.level.value > pos2.level.value else "DOWN"
        stairs_info.append((pos2, direction, pos1))

print("🪜 Available stairs on GROUND:")
for pos, direction, dest in stairs_info:
    print(f"  {pos.to_tuple()} -> {direction} to {dest.level.name} at {dest.to_tuple()}")

# Test 1: Navigate to UP stairs (to mountain)
up_stairs = [s for s in stairs_info if s[1] == "UP"]
if up_stairs:
    target, _, dest = up_stairs[0]
    print(f"\n🎯 Test 1: Going to MOUNTAIN stairs at {target.to_tuple()}")
    
    # Navigate
    moves = 0
    while sailor.position != target and moves < 50:
        dx = target.x - sailor.position.x
        dy = target.y - sailor.position.y
        
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        else:
            break
        
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        
        if not info[PLAYER_ID].get('success'):
            # Try perpendicular
            if abs(dy) > abs(dx) and dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            elif dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            else:
                break
            action = Action(sailor_id=PLAYER_ID, action_type=action_type)
            env.step({PLAYER_ID: action})
        
        moves += 1
    
    print(f"✅ Reached stairs in {moves} moves")
    show_sailor_status()
    
    # Climb UP
    print(f"\n🧗 Climbing UP to MOUNTAIN...")
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.CLIMB_UP)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    print(f"Result: {info[PLAYER_ID]}")
    show_sailor_status()
    
    # Move on mountain
    print(f"\n🚶 Moving EAST 3 times on MOUNTAIN...")
    for i in range(3):
        action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_EAST)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        print(f"  Step {i+1}: {sailor.position.to_tuple()}")
    
    print(f"\n📍 Final position: {sailor.position.to_tuple()}")
    print(f"✅ SUCCESS: Alice is on {sailor.position.level.name} level")

🧪 TEST: Multi-Level Navigation
Starting position: (15, 15, <MapLevel.GROUND: 0>)

🪜 Available stairs on GROUND:
  (10, 19, <MapLevel.GROUND: 0>) -> UP to MOUNTAIN at (0, 0, <MapLevel.MOUNTAIN: 2>)
  (15, 16, <MapLevel.GROUND: 0>) -> DOWN to CAVE at (0, 0, <MapLevel.CAVE: -1>)

🎯 Test 1: Going to MOUNTAIN stairs at (10, 19, <MapLevel.GROUND: 0>)
✅ Reached stairs in 9 moves
📍 Position: (10, 19, <MapLevel.GROUND: 0>)
⚡ Energy: 91/100
🎒 Backpack: {}
--------------------------------------------------

🧗 Climbing UP to MOUNTAIN...
Result: {'success': True, 'new_level': 'MOUNTAIN', 'new_position': (0, 0, <MapLevel.MOUNTAIN: 2>), 'energy_cost': 3, 'alive': True, 'is_traitor': False}
📍 Position: (0, 0, <MapLevel.MOUNTAIN: 2>)
⚡ Energy: 88/100
🎒 Backpack: {}
--------------------------------------------------

🚶 Moving EAST 3 times on MOUNTAIN...
  Step 1: (1, 0, <MapLevel.MOUNTAIN: 2>)
  Step 2: (2, 0, <MapLevel.MOUNTAIN: 2>)
  Step 3: (3, 0, <MapLevel.MOUNTAIN: 2>)

📍 Final position: (3, 0, <Ma

In [15]:
# Display all three levels
print(env.render_map(MapLevel.GROUND, use_emoji=True))
print(env.render_map(MapLevel.MOUNTAIN, use_emoji=True))
print(env.render_map(MapLevel.CAVE, use_emoji=True))


🏝️  GROUND LEVEL (Z=0)
Legend: 🟫 land | 🌲 wood | ⚙️ metal | 🍎 food | 🌿 antidote | ☠️ poison
        ⬆️ stairs up | ⬇️ stairs down | 🏠 base | A/B/C/D/E sailors | 5👥 group

   012345678901234567890123456789
 0 🟫🟫🟫🟫🍎🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫
 1 🍎🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫
 2 🟫🟫🟫🟫🌲🟫🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 3 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫⚙️🟫⚙️🟫⚙️🍎🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫
 4 🟫🟫🌲🟫🍎🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫
 5 🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🌲🟫🟫🟫
 6 🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🍎🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 7 🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🍎🟫🟫🟫🟫
 8 ⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🍎🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🌲
 9 🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🌲🌲
10 🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🍎🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
11 🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🌲🟫🟫🌲🟫
12 🟫🍎🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫🟫⚙️🟫🟫🟫🟫
13 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
14 🟫🟫🟫🌲🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫🟫
15 🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫4👥🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫
16 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🍎🍎⬇️🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
17 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫⚙️🟫
18 🟫🌲🟫🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
19 🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫⬆️🟫🟫🟫🟫🍎🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫⚙️
20 🟫🟫⚙️🟫🌲🟫🍎⚙️🟫⚙️🟫🟫🟫🍎☠️🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫⚙️🟫🟫🟫
21 🟫🟫🍎🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🌲🟫🟫🍎🟫🟫
22 🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫

In [16]:
# TEST 2: Navigate back down and then to CAVE

print("\n" + "=" * 70)
print("🧪 TEST 2: Navigate to CAVE")
print("=" * 70)

# First, find stairs down on mountain
sailor = env.state.sailors[PLAYER_ID]
current_level = sailor.position.level

mountain_stairs = []
for pos1, pos2 in env.state.world_map.level_transitions:
    if pos1.level == current_level:
        direction = "UP" if pos2.level.value > pos1.level.value else "DOWN"
        mountain_stairs.append((pos1, direction, pos2))
    if pos2.level == current_level:
        direction = "UP" if pos1.level.value > pos2.level.value else "DOWN"
        mountain_stairs.append((pos2, direction, pos1))

print(f"Current position: {sailor.position.to_tuple()}")
print(f"Stairs on {current_level.name}:")
for pos, direction, dest in mountain_stairs:
    print(f"  {pos.to_tuple()} -> {direction} to {dest.level.name}")

# Navigate to stairs
down_stairs = [s for s in mountain_stairs if s[1] == "DOWN"]
if down_stairs:
    target, _, _ = down_stairs[0]
    print(f"\n🎯 Going to stairs at {target.to_tuple()}")
    
    moves = 0
    while sailor.position != target and moves < 30:
        dx = target.x - sailor.position.x
        dy = target.y - sailor.position.y
        
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        else:
            break
        
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        
        if not info[PLAYER_ID].get('success'):
            if abs(dy) > abs(dx) and dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            elif dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            else:
                break
            action = Action(sailor_id=PLAYER_ID, action_type=action_type)
            env.step({PLAYER_ID: action})
        
        moves += 1
    
    print(f"✅ Reached in {moves} moves")
    
    # Climb down to ground
    print(f"\n🧗 Climbing DOWN to GROUND...")
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.CLIMB_DOWN)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    print(f"Result: {info[PLAYER_ID]}")
    print(f"Position: {sailor.position.to_tuple()}")

# Now go to CAVE stairs
ground_stairs = []
for pos1, pos2 in env.state.world_map.level_transitions:
    if pos1.level == MapLevel.GROUND:
        direction = "UP" if pos2.level.value > pos1.level.value else "DOWN"
        ground_stairs.append((pos1, direction, pos2))
    if pos2.level == MapLevel.GROUND:
        direction = "UP" if pos1.level.value > pos2.level.value else "DOWN"
        ground_stairs.append((pos2, direction, pos1))

cave_stairs = [s for s in ground_stairs if s[1] == "DOWN"]
if cave_stairs:
    target, _, _ = cave_stairs[0]
    print(f"\n🎯 Going to CAVE stairs at {target.to_tuple()}")
    
    moves = 0
    while sailor.position != target and moves < 50:
        dx = target.x - sailor.position.x
        dy = target.y - sailor.position.y
        
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        else:
            break
        
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        
        if not info[PLAYER_ID].get('success'):
            if abs(dy) > abs(dx) and dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            elif dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            else:
                break
            action = Action(sailor_id=PLAYER_ID, action_type=action_type)
            env.step({PLAYER_ID: action})
        
        moves += 1
    
    print(f"✅ Reached in {moves} moves")
    
    # Climb down to cave
    print(f"\n🧗 Climbing DOWN to CAVE...")
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.CLIMB_DOWN)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    print(f"Result: {info[PLAYER_ID]}")
    print(f"Position: {sailor.position.to_tuple()}")
    
    # Move in cave
    print(f"\n🚶 Moving SOUTH 3 times in CAVE...")
    for i in range(3):
        action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_SOUTH)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        print(f"  Step {i+1}: {sailor.position.to_tuple()}")

print(f"\n✅ Final position: {sailor.position.to_tuple()}")
if sailor.position.level == MapLevel.CAVE:
    print("✅ TEST PASSED: Alice is in CAVE level!")
else:
    print(f"❌ TEST FAILED: Alice is on {sailor.position.level.name} level!")


🧪 TEST 2: Navigate to CAVE
Current position: (3, 0, <MapLevel.MOUNTAIN: 2>)
Stairs on MOUNTAIN:
  (0, 0, <MapLevel.MOUNTAIN: 2>) -> DOWN to GROUND

🎯 Going to stairs at (0, 0, <MapLevel.MOUNTAIN: 2>)
✅ Reached in 3 moves

🧗 Climbing DOWN to GROUND...
Result: {'success': True, 'new_level': 'GROUND', 'new_position': (10, 19, <MapLevel.GROUND: 0>), 'energy_cost': 1, 'alive': True, 'is_traitor': False}
Position: (10, 19, <MapLevel.GROUND: 0>)

🎯 Going to CAVE stairs at (15, 16, <MapLevel.GROUND: 0>)
✅ Reached in 8 moves

🧗 Climbing DOWN to CAVE...
Result: {'success': True, 'new_level': 'CAVE', 'new_position': (0, 0, <MapLevel.CAVE: -1>), 'energy_cost': 1, 'alive': True, 'is_traitor': False}
Position: (0, 0, <MapLevel.CAVE: -1>)

🚶 Moving SOUTH 3 times in CAVE...
  Step 1: (0, 1, <MapLevel.CAVE: -1>)
  Step 2: (0, 2, <MapLevel.CAVE: -1>)
  Step 3: (0, 3, <MapLevel.CAVE: -1>)

✅ Final position: (0, 3, <MapLevel.CAVE: -1>)
✅ TEST PASSED: Alice is in CAVE level!


In [17]:
# Display all three levels
print(env.render_map(MapLevel.GROUND, use_emoji=True))
print(env.render_map(MapLevel.MOUNTAIN, use_emoji=True))
print(env.render_map(MapLevel.CAVE, use_emoji=True))


🏝️  GROUND LEVEL (Z=0)
Legend: 🟫 land | 🌲 wood | ⚙️ metal | 🍎 food | 🌿 antidote | ☠️ poison
        ⬆️ stairs up | ⬇️ stairs down | 🏠 base | A/B/C/D/E sailors | 5👥 group

   012345678901234567890123456789
 0 🟫🟫🟫🟫🍎🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫
 1 🍎🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫
 2 🟫🟫🟫🟫🌲🟫🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 3 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫⚙️🟫⚙️🟫⚙️🍎🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫
 4 🟫🟫🌲🟫🍎🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫
 5 🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🌲🟫🟫🟫
 6 🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🍎🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 7 🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🍎🟫🟫🟫🟫
 8 ⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🍎🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🌲
 9 🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🌲🌲
10 🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🍎🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
11 🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🌲🟫🟫🌲🟫
12 🟫🍎🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫🟫⚙️🟫🟫🟫🟫
13 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
14 🟫🟫🟫🌲🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫🟫
15 🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫4👥🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫
16 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🍎🍎⬇️🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
17 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫⚙️🟫
18 🟫🌲🟫🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
19 🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫⬆️🟫🟫🟫🟫🍎🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫⚙️
20 🟫🟫⚙️🟫🌲🟫🍎⚙️🟫⚙️🟫🟫🟫🍎☠️🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫⚙️🟫🟫🟫
21 🟫🟫🍎🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🌲🟫🟫🍎🟫🟫
22 🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫

## 🤖 AI Agent Navigation - Boundary Testing

Testing boundary enforcement to ensure AI agents cannot move outside the map bounds.

In [18]:
# TEST: Boundary Enforcement
print("=" * 70)
print("🧪 BOUNDARY ENFORCEMENT TEST")
print("=" * 70)

# Reset environment for clean test
env.reset(seed=42)
sailor = env.state.sailors[PLAYER_ID]

print(f"\n📍 Starting position: {sailor.position.to_tuple()}")
print(f"Map size for GROUND level: 30x30 (indices 0-29)")

# Test 1: Try to move beyond NORTH boundary (y=0)
print("\n" + "-" * 70)
print("Test 1: Moving to NORTH boundary (y=0)")
print("-" * 70)

# Move to top edge
while sailor.position.y > 0:
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_NORTH)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    if not info[PLAYER_ID].get('success'):
        print(f"❌ Blocked at y={sailor.position.y}: {info[PLAYER_ID].get('reason')}")
        break

print(f"Position at boundary: {sailor.position.to_tuple()}")

# Try to go beyond
action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_NORTH)
obs, _, _, _, info = env.step({PLAYER_ID: action})
print(f"\n🚫 Trying to move NORTH from y=0:")
print(f"   Success: {info[PLAYER_ID].get('success')}")
print(f"   Reason: {info[PLAYER_ID].get('reason')}")
print(f"   Position after: {sailor.position.to_tuple()}")
if sailor.position.y == 0:
    print(f"   ✅ PASS: Agent stayed at boundary y=0")

# Test 2: Try to move beyond WEST boundary (x=0)
print("\n" + "-" * 70)
print("Test 2: Moving to WEST boundary (x=0)")
print("-" * 70)

while sailor.position.x > 0:
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_WEST)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    if not info[PLAYER_ID].get('success'):
        break

print(f"Position at boundary: {sailor.position.to_tuple()}")

action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_WEST)
obs, _, _, _, info = env.step({PLAYER_ID: action})
print(f"\n🚫 Trying to move WEST from x=0:")
print(f"   Success: {info[PLAYER_ID].get('success')}")
print(f"   Reason: {info[PLAYER_ID].get('reason')}")
print(f"   Position after: {sailor.position.to_tuple()}")
if sailor.position.x == 0:
    print(f"   ✅ PASS: Agent stayed at boundary x=0")

# Test 3: Try SOUTH boundary (y=29 for 30x30 map)
print("\n" + "-" * 70)
print("Test 3: Moving to SOUTH boundary (y=29)")
print("-" * 70)

while sailor.position.y < 29:
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_SOUTH)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    if not info[PLAYER_ID].get('success'):
        break

print(f"Position at boundary: {sailor.position.to_tuple()}")

action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_SOUTH)
obs, _, _, _, info = env.step({PLAYER_ID: action})
print(f"\n🚫 Trying to move SOUTH from y=29:")
print(f"   Success: {info[PLAYER_ID].get('success')}")
print(f"   Reason: {info[PLAYER_ID].get('reason')}")
print(f"   Position after: {sailor.position.to_tuple()}")
if sailor.position.y == 29:
    print(f"   ✅ PASS: Agent stayed at boundary y=29")

# Test 4: Try EAST boundary (x=29 for 30x30 map)
print("\n" + "-" * 70)
print("Test 4: Moving to EAST boundary (x=29)")
print("-" * 70)

while sailor.position.x < 29:
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_EAST)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    if not info[PLAYER_ID].get('success'):
        break

print(f"Position at boundary: {sailor.position.to_tuple()}")

action = Action(sailor_id=PLAYER_ID, action_type=ActionType.MOVE_EAST)
obs, _, _, _, info = env.step({PLAYER_ID: action})
print(f"\n🚫 Trying to move EAST from x=29:")
print(f"   Success: {info[PLAYER_ID].get('success')}")
print(f"   Reason: {info[PLAYER_ID].get('reason')}")
print(f"   Position after: {sailor.position.to_tuple()}")
if sailor.position.x == 29:
    print(f"   ✅ PASS: Agent stayed at boundary x=29")

print("\n" + "=" * 70)
print("✅ ALL BOUNDARY TESTS PASSED!")
print("AI agents CANNOT move outside map bounds.")
print("=" * 70)

🧪 BOUNDARY ENFORCEMENT TEST

📍 Starting position: (15, 15, <MapLevel.GROUND: 0>)
Map size for GROUND level: 30x30 (indices 0-29)

----------------------------------------------------------------------
Test 1: Moving to NORTH boundary (y=0)
----------------------------------------------------------------------
Position at boundary: (15, 0, <MapLevel.GROUND: 0>)

🚫 Trying to move NORTH from y=0:
   Success: False
   Reason: Position not walkable
   Position after: (15, 0, <MapLevel.GROUND: 0>)
   ✅ PASS: Agent stayed at boundary y=0

----------------------------------------------------------------------
Test 2: Moving to WEST boundary (x=0)
----------------------------------------------------------------------
Position at boundary: (0, 0, <MapLevel.GROUND: 0>)

🚫 Trying to move WEST from x=0:
   Success: False
   Reason: Position not walkable
   Position after: (0, 0, <MapLevel.GROUND: 0>)
   ✅ PASS: Agent stayed at boundary x=0

----------------------------------------------------------

## 🤖 AI Agent Action Examples

Examples of how AI agents will use the available actions.

In [10]:
# Example: How AI agents will interact with the environment

print("=" * 70)
print("🤖 AI AGENT EXAMPLE: Simple Navigation")
print("=" * 70)

# Reset environment
env.reset(seed=42)

# Simulate AI agent decision-making
print("\n📋 AI Agent: 'Alice' decides to navigate to position (20, 20)")

# AI agent's simple navigation function
def ai_navigate_to_target(env, agent_id, target_x, target_y, max_steps=20):
    """Simple AI navigation function"""
    steps = 0
    
    while steps < max_steps:
        sailor = env.state.sailors[agent_id]
        current_x, current_y = sailor.position.x, sailor.position.y
        
        # Check if at target
        if current_x == target_x and current_y == target_y:
            print(f"✅ AI reached target in {steps} steps!")
            return True
        
        # Calculate direction
        dx = target_x - current_x
        dy = target_y - current_y
        
        # Decide action (simple greedy approach)
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            direction_name = "EAST" if dx > 0 else "WEST"
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            direction_name = "SOUTH" if dy > 0 else "NORTH"
        else:
            break
        
        # AI creates action
        action = Action(sailor_id=agent_id, action_type=action_type)
        
        # Environment executes action
        obs, reward, done, truncated, info = env.step({agent_id: action})
        
        # AI processes result
        if info[agent_id]['success']:
            print(f"  Step {steps+1}: Moved {direction_name} → Position: ({sailor.position.x}, {sailor.position.y})")
        else:
            # Movement failed - try alternative direction
            print(f"  Step {steps+1}: {direction_name} blocked, trying alternative...")
            
            # Try perpendicular direction
            if abs(dy) > abs(dx) and dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            elif dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            else:
                print("  ❌ Stuck! Cannot reach target.")
                return False
            
            action = Action(sailor_id=agent_id, action_type=action_type)
            obs, reward, done, truncated, info = env.step({agent_id: action})
            
            if not info[agent_id]['success']:
                print("  ❌ Both directions blocked! Navigation failed.")
                return False
        
        steps += 1
    
    print(f"❌ Max steps reached without reaching target")
    return False

# Run AI navigation
target_x, target_y = 20, 20
result = ai_navigate_to_target(env, PLAYER_ID, target_x, target_y, max_steps=20)

sailor = env.state.sailors[PLAYER_ID]
print(f"\n📍 Final position: ({sailor.position.x}, {sailor.position.y})")
print(f"⚡ Energy remaining: {sailor.energy}/{MAX_ENERGY}")

print("\n" + "=" * 70)
print("✅ AI agent successfully navigated using the action system!")
print("✅ Boundary checking prevented any out-of-bounds moves")
print("✅ Energy was automatically managed by the environment")
print("=" * 70)

🤖 AI AGENT EXAMPLE: Simple Navigation

📋 AI Agent: 'Alice' decides to navigate to position (20, 20)
  Step 1: Moved SOUTH → Position: (15, 16)
  Step 2: Moved EAST → Position: (16, 16)
  Step 3: Moved SOUTH → Position: (16, 17)
  Step 4: Moved EAST → Position: (17, 17)
  Step 5: Moved SOUTH → Position: (17, 18)
  Step 6: Moved EAST → Position: (18, 18)
  Step 7: Moved SOUTH → Position: (18, 19)
  Step 8: Moved EAST → Position: (19, 19)
  Step 9: Moved SOUTH → Position: (19, 20)
  Step 10: Moved EAST → Position: (20, 20)
✅ AI reached target in 10 steps!

📍 Final position: (20, 20)
⚡ Energy remaining: 90/100

✅ AI agent successfully navigated using the action system!
✅ Boundary checking prevented any out-of-bounds moves
✅ Energy was automatically managed by the environment
